In [5]:
import pandas as pd
import yfinance as yf 
from sqlalchemy import create_engine
import talib as ta
engine = create_engine('sqlite:///../../dataset/us/us_ticker_list_with_name.db')

In [ ]:
ticker_df = pd.read_sql('TOTAL',con=engine)
ticker_list = ticker_df.Symbol.to_list()
ticker_str = ''
for ticker in ticker_list:
    ticker_str = ticker_str + str(ticker) + " "
# download data    
data = yf.download(
    tickers = ticker_str,
    # tickers = 'AAPL BABA AMZN',
    period = '2y',
    interval = '1d',
    group_by = 'ticker',
    auto_adjust = True,
    treads = True
)

# 关闭警告的设置方法
pd.set_option('mode.chained_assignment', None)
error_list = []
total = pd.DataFrame(columns=['Open', 'High', 'Low', 'Close', 'Volume', 'datetime', 'MA10', 'MA20',
       'MA50', 'MA150', 'MA200', 'VMA20', 'VMA50', 'symbol'])
for ticker in ticker_list:
    try:
        print('{} start'.format(ticker))
        df = data['{}'.format(ticker)]
        # 去除空行
        df.dropna(axis=0, inplace=True)
        df['symbol'] = ticker
        df['datetime'] = df.index
        timeperiod = [10,20,50,150,200]
        for period in timeperiod:
            df['MA{}'.format(period)] = ta.SMA(df.Close, timeperiod=period)
        vol_timeperiod = [20,50]
        for period in vol_timeperiod:
            df['VMA{}'.format(period)] = ta.SMA(df.Volume, timeperiod=period)
        df['52week_high'] = df.tail(252).High.max()
        df['52week_low'] = df.tail(252).Low.min()

        df = df.tail(252)
        df['rs'] = round((df.tail(1).Close.values[0] - df.head(1).Close.values[0]) / df.head(1).Close.values[0],3)

        target = df.tail(1)
        total = pd.concat([total, target],ignore_index=True)
    except:
        print('{} error'.format(ticker))
        error_list.append(ticker)
total.dropna(inplace=True)
total.index = range(len(total))
total.datetime = total.datetime.apply(lambda x:str(x)[:10])

In [7]:
engine = create_engine('sqlite:///../../dataset/us/us_ticker_with_indicator.db')

In [ ]:
datetime = total.iloc[1].datetime
total.to_sql('{}'.format(datetime),if_exists='replace', con=engine, index=None)

6121